In [ ]:
# res = compare_experiments_barplot(
#     experiment_paths=[experiment_output_dir],
#     title="TARS eval.",
# )


## WANDB dev

In [ ]:
import numpy as np
import pandas as pd
import wandb
from skmultilearn.model_selection import iterative_train_test_split


def label_dictionary_to_label_mat(label_dictionary_list, thresh=0.75):
    return (
        pd.DataFrame.from_records(list(label_dictionary_list))
        .pipe(lambda x: x >= thresh)
        .astype(int)
    )


def label_mat_to_label_dictionary(label_mat):
    return list(label_mat.to_dict(orient="index").values())


def create_multi_label_train_test_splits(
    df: pd.core.frame.DataFrame,
    label_col: str,
    test_size=0.25,
):
    df[label_col] = df[label_col].apply(
        lambda x: eval(x) if type(x) == str else x
    )  # string > dict

    # threshold, iteratively split
    y_df = label_dictionary_to_label_mat(df[label_col])
    y_cols = list(y_df.columns)
    x_df = df.drop(label_col, axis=1)
    x_cols = list(x_df.columns)

    x_train, y_train, x_test, y_test = iterative_train_test_split(
        x_df.values, y_df.astype(int).values, test_size=test_size
    )

    # convert back to label object form
    y_train = label_mat_to_label_dictionary(pd.DataFrame(y_train, columns=y_cols))
    y_test = label_mat_to_label_dictionary(pd.DataFrame(y_test, columns=y_cols))

    # re-stack x/y
    train = pd.DataFrame(np.column_stack((x_train, y_train))).set_axis(
        labels=x_cols + [label_col], axis="columns", inplace=False
    )

    test = pd.DataFrame(np.column_stack((x_test, y_test))).set_axis(
        labels=x_cols + [label_col], axis="columns", inplace=False
    )
    return train, test


In [ ]:
from pathlib import Path

import pandas as pd
import yaml

CONFIG = yaml.safe_load(
    Path(
        "/Users/samhardyhey/Desktop/blog/blog-multi-label/training_config.yaml"
    ).read_bytes()
)

# 1.1 create splits
df = pd.read_csv(CONFIG["dataset"])
train, test = create_multi_label_train_test_splits(
    df, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)
test, dev = create_multi_label_train_test_splits(
    test, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)


In [ ]:
# 1.2 log splits
with wandb.init(
    project=CONFIG["wandb_project"],
    name="dataset",
    group=CONFIG["wandb_group"],
    entity="cool_stonebreaker",
) as run:
    dataset_artifact = wandb.Artifact(
        "reddit-aus-finance", type="dataset", description="Train, dev, test splits"
    )
    dataset_artifact.add(wandb.Table(dataframe=train), name="train")
    dataset_artifact.add(wandb.Table(dataframe=dev), name="dev")
    dataset_artifact.add(wandb.Table(dataframe=test), name="test")
    run.log_artifact(dataset_artifact)


In [ ]:
from clear_bow.classifier import DictionaryClassifier

model_config = CONFIG["models"][0]

dc = DictionaryClassifier(
    classifier_type=model_config["classifier_type"],
    label_dictionary=model_config["label_dictionary"],
)


In [ ]:
# get train/test performance > log as model artefacts?
dev_pred = dev.assign(pred=lambda x: x[CONFIG["text_col"]].apply(dc.predict_single))
test_pred = test.assign(pred=lambda x: x[CONFIG["text_col"]].apply(dc.predict_single))


In [ ]:
from sklearn.metrics import classification_report

label_names = label_dictionary_to_label_mat(test_pred).columns.tolist()
class_report = classification_report(
    label_dictionary_to_label_mat(test[CONFIG["label_col"]]),
    label_dictionary_to_label_mat(test_pred),
    target_names=label_names,
    output_dict=True,
)

# reduce output into wandb?
slim_class_report = (
    pd.DataFrame(class_report)
    .T.reset_index()
    .pipe(lambda x: x[x["index"].isin(label_names)])
    .pipe(lambda x: x[["index", "f1-score", "support"]])
    .rename(mapper={"index": "label"}, axis="columns", inplace=False)
    .set_index("label")
    .to_dict(orient="index")
)


In [ ]:
# mock seperate model runs, within the same group
with wandb.init(
    project=CONFIG["wandb_project"],
    name="dictionary_classifier",
    group=CONFIG["wandb_group"],
    entity="cool_stonebreaker",
) as run:
    wandb.config.model = model_config["model"]

    dev_preds_artifact = wandb.Artifact(
        "dev_preds", type="dataset", description="Dev predictions"
    )
    dataset_artifact.add(wandb.Table(dataframe=dev_pred), name="dev_preds")

    wandb.log(slim_class_report)
    wandb.summary["test_f1"] = class_report["weighted avg"]["f1-score"]
    wandb.summary["test_support"] = class_report["weighted avg"]["support"]

# seperate models as seperate runs
with wandb.init(
    project=CONFIG["wandb_project"],
    name="flair_tars",
    group=CONFIG["wandb_group"],
    entity="cool_stonebreaker",
) as run:
    wandb.config.model = "flair_tars"

    dev_preds_artifact = wandb.Artifact(
        "dev_preds", type="dataset", description="Dev predictions"
    )
    dataset_artifact.add(wandb.Table(dataframe=dev_pred), name="dev_preds")

    wandb.log(slim_class_report)
    wandb.summary["test_f1"] = class_report["weighted avg"]["f1-score"]
    wandb.summary["test_support"] = class_report["weighted avg"]["support"]
    wandb.finish()


In [ ]:
# clear out for dev purposes
import wandb

api = wandb.Api()

for run in api.runs(path="cool_stonebreaker/tyre_kick"):
    run = api.run(f"cool_stonebreaker/tyre_kick/{run.id}")
    run.delete()
